In [1]:
import os
import re
import gensim
from gensim.test.utils import common_texts
from gensim.models import FastText
from gensim.test.utils import get_tmpfile
import numpy as np
from gensim.models import KeyedVectors

# TRAIN FASTTEXT EMBEDDINGS

In [2]:
from gensim.models.word2vec import LineSentence
from gensim.models.fasttext import FastText as FT_gensim

# Set file names for train and test data
data_dir = '/home/ashimko/Documents/HSE/Krylov/Lojban/lojban_corpus/'
lee_train_file = data_dir + 'lojban_corpus.txt'
lee_data = LineSentence(lee_train_file)

embedding_size = 300
model_gensim = FT_gensim(size=embedding_size)

# build the vocabulary
model_gensim.build_vocab(lee_data)

# train the model
model_gensim.train(lee_data, total_examples=model_gensim.corpus_count, epochs=model_gensim.iter)

print(model_gensim)

/home/ashimko/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  app.launch_new_instance()


FastText(vocab=2046, size=300, alpha=0.025)


In [3]:
embedding_path = '/media/ashimko/data_storage/EMBEDDINGS/'
model_gensim.save(embedding_path + 'lojban_300d.vec')
model_gensim['mi'].shape

/home/ashimko/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(300,)

In [10]:
list(lojban_words)

["no'i",
 'la',
 'alis',
 "co'a",
 'tatpi',
 'le',
 'nu',
 'zutse',
 'rirxe',
 'korbi',
 "re'o",
 'mensi',
 "gi'e",
 'zukte',
 'fi',
 'noda',
 'i',
 'sutra',
 'zgana',
 'cukta',
 'poi',
 'cu',
 'tcidu',
 "ku'i",
 'cy',
 'vasru',
 'no',
 'pixra',
 'ja',
 'nuncasnu',
 'lu',
 "ji'a",
 'ma',
 'prali',
 'sei',
 'pensi',
 'lo',
 "li'u",
 'abu',
 "ca'o",
 'menli',
 'jdice',
 'to',
 'xagrai',
 'pe',
 "va'o",
 'glare',
 'djedi',
 'rinka',
 'lifri',
 'je',
 'bebna',
 'toi',
 "du'u",
 'xukau',
 'pluka',
 'fa',
 'zbasu',
 'xrula',
 'linsi',
 'se',
 'vamji',
 'raktu',
 'crepu',
 'loi',
 'suksa',
 'blabi',
 'ractu',
 'xunblabi',
 'kanla',
 'bajra',
 "la'e",
 "di'u",
 "no'e",
 "ba'e",
 'mutce',
 'ka',
 'cizra',
 'jinvi',
 'kei',
 'tirna',
 'sezysku',
 'mi',
 'lerci',
 'baku',
 'ca',
 'ri',
 'te',
 'sidbo',
 'ei',
 'caku',
 'piro',
 'simlu',
 'rarna',
 "ca'a",
 'lebna',
 'junla',
 'kosta',
 'daski',
 'catlu',
 'jy',
 "di'a",
 'spaji',
 "ki'u",
 'lindi',
 'pagre',
 "si'o",
 'pu',
 'noroi',
 'viska',
 '

In [23]:
max([len(word.split()) for word in lojban_words])

1

In [47]:
lojban_words = list(model_gensim.wv.vocab.keys())
loj_embedding = '{} {}\n'.format(len(lojban_words), embedding_size)

for word in lojban_words:
    loj_embedding += word + ' ' + ' '.join(map(str, model_gensim[word])) + '\n'

with open('lojban_300d.vec', 'w') as f:
    f.write(loj_embedding)

/home/ashimko/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


# BUILD ENGLISH-LOJBAN DICTIONARY

Добавляем только слова 1-1

### INDEX - ONE ENGLISH WORD

In [38]:
dict_path = '/home/ashimko/Documents/HSE/Krylov/ralju/eng-jbo_basic-english-wordlist.dsl'
eng_loj_dict = {}
with open(dict_path, 'r') as f:
    lines = f.readlines()
    for i in range(4, len(lines)):
        one_to_one_flag = False
        try:
            if lines[i][0] != '\t' and lines[i+2] != '\t':
                if len(lines[i].split()) == 1 and len(lines[i+1].split(',')) == 1:
                    key = lines[i].strip()
                    value = lines[i+1]
                    value = re.sub(r'\([^()]*\)', '', value)
                    value = re.sub(r'\[[^()]*\]', '', value).strip()
                    value = value.split('/')[0]
                    if len(value) > 0:
                        eng_loj_dict[key] = value
        except:
            print(i)
            continue

eng_loj_dict['as'] = 'du\'i'
eng_loj_dict.pop('condition')
# eng_loj_dict.pop("??? ('as she rounds the corner,...' -- ie movement path )")
eng_loj_dict.pop('together')

print('LENGTH DICT', len(eng_loj_dict))

1769
LENGTH DICT 664


### INDEX - SEVERAL ENGLISH WORDS

In [6]:
# dict_path = '/home/ashimko/Documents/HSE/Krylov/ralju/eng-jbo_cmavo-keywords.dsl'
# eng_loj_dict = {}
# with open(dict_path, 'r') as f:
#     lines = f.readlines()
#     for i in range(4, len(lines)):
#         one_to_one_flag = False
#         try:
#             if lines[i][0] != '\t' and lines[i+2] != '\t':
#                 if len(lines[i+1].split(',')) == 1:
#                     key = lines[i].strip()
#                     value = lines[i+1].replace('>', '').replace('<', '').strip()
#                     if len(value) > 0:
#                         eng_loj_dict[key] = value
#         except:
#             print(i)
#             continue

# print('LENGTH DICT', len(eng_loj_dict))

2184
LENGTH DICT 1084


In [49]:
7 % 7 == 0

True

In [46]:
eng_loj_dict_train = ''
eng_loj_dict_test = ''

i = 0
for key, value in eng_loj_dict.items():
    if len(key.split()) == 1 and len(value.split()) == 1:
        if i % 7 != 0:
            eng_loj_dict_train += '{} {}\n'.format(key.lower(), value.lower())
        else:
            eng_loj_dict_test += '{} {}\n'.format(key.lower(), value.lower())
        i+=1
    else:
        continue

with open('eng_loj_train.dict', 'w') as f:
    f.write(eng_loj_dict_train)
with open('eng_loj_test.dict', 'w') as f:
    f.write(eng_loj_dict_train)

In [52]:
len(eng_loj_dict_train.split('\n'))

521

In [53]:
len(eng_loj_dict_test.split('\n'))

88